# Documentación y Guía Maestra: Librería de Difusión y Peeling de Redes

Esta librería está diseñada para investigar la propagación de influencia en redes complejas utilizando un enfoque de **difusión de masa** y una técnica de optimización denominada **Peeling (Pelado)**. 

El objetivo principal es identificar nodos críticos (semillas) y comparar la eficacia del algoritmo de Pelado contra estándares de la industria como **CELF** (Cost-Effective Lazy Forward) y **RIS** (Reverse Influence Sampling).

In [1]:
import networkx as nx
import pandas as pd
import os
from difusion_lib import (
    GeneradorRedes, 
    MotorDifusion, 
    ControladorPelado, 
    ProcesadorSimulaciones, 
    ConvertidorGrafos
)

print("Librería cargada exitosamente.")

Librería cargada exitosamente.


## 1. Generación de Topologías (`GeneradorRedes`)

La clase `GeneradorRedes` proporciona métodos estáticos para crear grafos con propiedades estructurales específicas.

### Métodos disponibles:

1. **`generar_cascada_estricta`**
   - `n_bloques` (int): Número de clústeres o grupos secuenciales.
   - `nodos_por_bloque` (int): Cantidad de nodos dentro de cada grupo.
   
2. **`generar_flujo_libre_escala`**
   - `n_nodos` (int): Tamaño total de la red con distribución de grado de ley de potencia.

3. **`generar_malla_estocastica_netlogo`**
   - `dim` (int): Dimensión de la rejilla (dim x dim).
   - `link_chance` (float 0-100): Probabilidad de que exista una conexión entre vecinos.

4. **`generar_red_social_realista`**
   - `n_users` (int): Nodos totales.
   - `m_neighbors` (int): Aristas a añadir por cada nodo nuevo.
   - `p_triangle` (float 0-1): Probabilidad de formar triángulos (clustering).
   - `ratio_mutual` (float 0-1): Probabilidad de que una arista sea bidireccional.

In [2]:
# Ejemplo: Generación de una red social con alta reciprocidad
G = GeneradorRedes.generar_red_social_realista(
    n_users=200, 
    m_neighbors=3, 
    p_triangle=0.2, 
    ratio_mutual=0.15
)

print(f"Grafo creado: {G.number_of_nodes()} nodos y {G.number_of_edges()} aristas.")

Grafo creado: 200 nodos y 500 aristas.


## 2. Simulación de Masa (`MotorDifusion`)

El `MotorDifusion` calcula cómo se transfiere la 'masa' o valor de influencia entre nodos basándose en una matriz de adyacencia normalizada.



### Parámetros:
- **Constructor (`__init__`)**:
    - `G`: Grafo de NetworkX.
    - `tasa_difusion` (float 0-1): Fracción de masa que un nodo reparte a sus vecinos en cada paso. El remanente (`1 - tasa`) se queda en el nodo.
- **Método `ejecutar`**:
    - `iteraciones` (int): Pasos de tiempo de la simulación. A mayor número, más cerca se estará del estado estacionario.

In [3]:
# Inicializar masa base en los nodos
for n in G.nodes():
    G.nodes[n]['val'] = 1.0

# Ejecutar motor
motor = MotorDifusion(G, tasa_difusion=0.6)
motor.ejecutar(iteraciones=100)

print("Masa difundida. Ejemplo de valor en nodo 0:", G.nodes[0]['val'])

Masa difundida. Ejemplo de valor en nodo 0: 4.05780313829486e-07


## 3. Proceso de Peeling (`ControladorPelado`)

Esta es la funcionalidad central. El controlador realiza ciclos de difusión y elimina nodos que superan un umbral de masa, permitiendo encontrar el "núcleo" de influencia.



### Parámetros de `ejecutar_estudio_pelado`:
- `num_pelados` (int): Máximo de capas de eliminación a realizar.
- `iteraciones_por_pelado` (int): Pasos de difusión antes de decidir qué nodos quitar.
- `umbral_masa` (float): Si un nodo tiene más masa que este valor, es eliminado.
- `umbral_nodos_final` (int): Condición de parada si la red se vuelve demasiado pequeña.
- `tasa_difusion` (float): Configuración para el motor interno.
- `valor_inicio` (float): Masa con la que reinician los nodos vivos en cada capa.
- `mostrar_graficos` (bool): Renderiza gráficos de matplotlib durante el proceso.
- `exportar_resultados` (bool): Guarda CSVs detallados.
- `carpeta_exportacion` (str): Ruta de destino.
- `generar_visualizaciones` (bool): Si es True, genera archivos HTML interactivos en 3D.
- `usar_cfc` (bool): Si es True, analiza Componentes Fuertemente Conexos en cada capa.

In [5]:
controlador = ControladorPelado(G)

_, figs_peel, G_survivors, pelados_dict = controlador.ejecutar_estudio_pelado(
    num_pelados=5, 
    umbral_masa=1.2, 
    umbral_nodos_final=2,
    iteraciones_por_pelado=100, 
    generar_visualizaciones=True,
    carpeta_exportacion="analisis_peeling_01"
)

print(f"Peeling completado. {len(figs_peel)} capas visualizables generadas.")

Iniciando Estudio: 200 nodos.
Pelado 1: Eliminando 50 componentes.
Pelado 2: Eliminando 7 componentes.
Pelado 3: Eliminando 9 componentes.
Pelado 4: Eliminando 8 componentes.
Pelado 5: Eliminando 27 componentes.
Peeling completado. 5 capas visualizables generadas.


## 4. Comparativa de Algoritmos (`ProcesadorSimulaciones`)

El `ProcesadorSimulaciones` permite ejecutar experimentos controlados comparando el Peeling contra CELF y RIS.

### Parámetros de `ejecutar_bateria_masiva`:
- `metodos` (list): Ej. `['pel', 'celf', 'ris']`.
- `configuraciones_grafos` (list): Lista de diccionarios con `tipo` y `params`.
- `n_simulaciones` (int): Cuántas veces repetir cada experimento para promediar.
- `master_folder` (str): Directorio para los reportes consolidados.
- `masa_total_concentrada` (float): Masa total asignada a las semillas elegidas para medir su impacto real de difusión.
- `tasa_difusion`, `num_pelados`, `umbral_masa`: Parámetros de control para el método de Pelado.

In [ ]:
simulador = ProcesadorSimulaciones()

config = [
    {'tipo': 'flujo_libre', 'params': {'n_nodos': 50}},
    {'tipo': 'malla_netlogo', 'params': {'dim': 20, 'link_chance': 60}}
]

simulador.ejecutar_bateria_masiva(
    metodos=['pel', 'ris'],
    configuraciones_grafos=config,
    generar_visualizaciones=True,
    generar_visualizaciones_pelado=True,
    n_simulaciones=1, 
    num_pelados=5,
    umbral_masa=1.1,
    master_folder="guia_2_resultados"
)

print("Batería masiva finalizada. Revisa la carpeta 'benchmark_resultados'.")


INICIANDO BATERÍA: FLUJO_LIBRE (1 sims)
Parámetros: {'n_nodos': 50}
>>> flujo_libre - Simulacion_001
Iniciando Estudio: 50 nodos.
Pelado 1: Eliminando 18 componentes.
Pelado 2: Eliminando 6 componentes.
Pelado 3: Eliminando 3 componentes.
Pelado 4: Eliminando 10 componentes.
Pelado 5: Eliminando 4 componentes.
Iniciando Difusión: 50 nodos.
Iniciando CELF...
Iniciando Difusión: 50 nodos.

INICIANDO BATERÍA: MALLA_NETLOGO (1 sims)
Parámetros: {'dim': 20, 'link_chance': 60}
>>> malla_netlogo - Simulacion_001
Iniciando Estudio: 400 nodos.
Pelado 1: Eliminando 83 componentes.
Pelado 2: Eliminando 44 componentes.
Pelado 3: Eliminando 51 componentes.
Pelado 4: Eliminando 41 componentes.
Pelado 5: Eliminando 38 componentes.
Iniciando Difusión: 400 nodos.
Iniciando CELF...
Iniciando Difusión: 400 nodos.

Generando reportes consolidados...
=== TODAS LAS BATERÍAS COMPLETADAS ===
Batería masiva finalizada. Revisa la carpeta 'benchmark_resultados'.


## 5. Visualización e Interoperabilidad

### Convertidor de Grafos
La librería permite pasar de NetworkX a iGraph (necesario para algoritmos optimizados como CELF):
- `ConvertidorGrafos.a_igraph(G_nx)`
- `ConvertidorGrafos.a_networkx(G_ig)`

### Dashboards
Si habilitaste `generar_visualizaciones`, podrás encontrar archivos `.html` en tu carpeta de resultados que muestran la red en 3D, donde el tamaño del nodo representa su masa acumulada.